<h3> Quick Summary </h3>
<p>Initially, an analysis was made of how the data would be and what are the possible inconsistencies with them.<br />

After the analysis is the treatment, in which changes were made that could bring inconsistencies to the views, such as the Release_Date column in its initial state.<br/>

After these two initial processes, we arrive at the views, where we have counts, max, min and other information that are relevant to insights.</p>

### Import

In [121]:
import pandas as pd

<h3> Analysis </h3>

In [86]:
df = pd.read_csv('../input/movies-list-of-all-american-movies-20082021/2008-2021_US_Movies.csv')

Creating a copy of the initial dataframe so as not to change the initial one directly;

Method:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html'>copy</a>

In [87]:
df_copy = df.copy()

In [88]:
df_copy

In [89]:
release_date_list = df_copy.Release_Date.unique().tolist()

In [90]:
df_copy.count()

In [ ]:
for i in release_date_list:
    print(i)

Checking for null or undefined values;

Methods:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html'>isna</a>: checks for nulls or undefined in the dataset;
2. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html'>sum</a>

In [92]:
df_copy.isna().sum()

In [93]:
try:
  for col in df_copy.columns:
      unique_values = len(df_copy[col].unique())
      print("Column '{col}' has '{unique_values}' unique values".format(col = col, unique_values=unique_values))
except Exception as e:
  print(str(e))

Checking for duplicate values;

Methods:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html'>duplicated</a>: duplicate checking;
2. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html'>sum</a>

In [94]:
df_copy.duplicated().sum()

Type checking;

Method:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html'>dtypes</a>: returns the types of dataframe columns;

In [95]:
df_copy.dtypes

<h3> Treatment </h3>

In [96]:
old_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
new_months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

Starting with changing the months in string to numbers, so that the column can be transformed into a datetime in the future;

Method:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html'>replace</a>: replace the specified value with the for loop;

In [97]:
cont = 0
for i in old_months:
    df_copy['Release_Date'] = df_copy.Release_Date.str.replace(f'{i}', f'{new_months[cont]}')
    cont += 1

In [98]:
df_copy

In [99]:
df_release_date = df_copy['Release_Date'].tolist()

As the use of replace was not working, it was necessary to use pure Python to change the dates, in addition to that, even with the use of replace, the years would be changed together with the days since they are the same in some lines - 12-Oct-12;

In [100]:
day = []
month = []
year = []
for i in df_release_date:
    if i[1:2] == '-':
        day.append('0' + i[0])
        month.append(i[2:4])
        year.append('20' + i[5:7])
    else:
        day.append(i[0:2])
        month.append(i[3:5])
        year.append('20' + i[6:8])

In [101]:
cont = 0
release_date_formated = []
for i in day:
    release_date_formated.append(day[cont] + '/' + month[cont] + '/' + year[cont])
    cont += 1

In [102]:
day = pd.DataFrame(day)
month = pd.DataFrame(month)
year = pd.DataFrame(year)

In [103]:
print('Checking that no data was lost during the transformation:', cont)

In [104]:
release_date_formated = pd.DataFrame(release_date_formated)

Renaming the column;

Method:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html'>rename</a>: rename the column

In [105]:
release_date_formated.rename(columns={0: 'Release_Date'}, inplace = True)

Changing column type to datetime;

Method:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html'>to_datetime</a>: to turn column into datetime type;

In [106]:
release_date_formated['Release_Date'] = pd.to_datetime(release_date_formated['Release_Date'])

The Release_Date column will be dropped in this situation because a new column with its information - changed - will be assigned to the dataframe;

Method:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html'>drop</a>: to drop the column;

In [107]:
df_copy.drop('Release_Date', axis = 1, inplace = True)

Assign the release_date_formated to my df_copy dataframe;

Method:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html'>assign</a>: assign new columns to a Dataframe;

In [108]:
df_copy = df_copy.assign(Release_Date = release_date_formated)

Assing the day, month and year dataframes to my df_copy dataframe;

Method:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html'>assign</a>: assign new columns to a Dataframe;

In [109]:
df_copy = df_copy.assign(Day = day)
df_copy = df_copy.assign(Month = month)
df_copy = df_copy.assign(Year = year)

<h3> Visualization </h3>

In this part of the code, I will bring information to answer questions that may arise.

*What is the genre/production company count?*

*What is the most number of times a genre/production company appears?*

*And the lowest values?*

With a lot of use of for, the respective insights and other information were answered.

Creation of lists to use in the for loop.

In [110]:
list_columns_visualization = ['Day', 'Month', 'Year']
list_target_columns = ['Genre', 'Production company']

In this first for loop, we will see information related 
to the target columns genre and production company;

With the help of the describe method, we can visualize the count, average, maximum and minimum of values, among other useful information.

Methods:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html'>value_counts</a>: used for counting;
2. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html'>describe</a>: generation of descriptive statistics;
3. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.median.html'>median</a>: median of the values;

<h5><font color='darkcyan'>What is the count, highest and lowest value per genre/Production company count?</font></h5>

In [111]:
for i in list_target_columns:
    print(f'{i}\n', df_copy[f'{i}'].value_counts().describe())
    print('median', df_copy[f'{i}'].value_counts().median(), '\n')

In the following for loop, we can view the highest and lowest counts - number of appearances in the dataset - by genre and production company;

The methods used were similar to the for past, with the addition of head and tail;

Methods:
1. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html'>head</a>: used to return the first 5 lines;
2. <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.tail.html'>tail</a>: used to return the last 5 lines;

<h5><font color='darkcyan'>The 5 highest and lowest count values ​​by genre/Production company</font></h5>

In [112]:
for i in list_target_columns:
    print(f'The 5 highest values | {i}\n', df_copy[f'{i}'].value_counts().head(5), '\n')
    print(f'The 5 lowest values | {i}\n', df_copy[f'{i}'].value_counts().tail(5), '\n')

In this for loop, the process extends, with the addition of another for loop;

What loops do in this usage is as follows:
for each value inside the list_columns_visualization - Day, Month, Year -, I want to be returned information related to the values that are in list_target_columns - Genre, Production Company.

I kept using the methods present in the for loops passed.

<h5><font color='darkcyan'>The 5 highest and lowest values ​​of genre/production company count by day, month, year</font></h5>

In [113]:
for i in list_columns_visualization:
    for j in list_target_columns:
        print(f'The 5 highest values | {i} | {j}\n', df_copy[[f'{i}', f'{j}']].value_counts().head(5), '\n')
        print(f'The 5 lowest values | {i} | {j}\n', df_copy[[f'{i}', f'{j}']].value_counts().tail(5), '\n')

My column for viewing in this case is the Release_Date to check which date had the most films produced by genre and by production company;

<h5><font color='darkcyan'>The 5 highest and lowest values ​​of genre/production company count by release date</font></h5>

In [114]:
for i in list_target_columns:   
    print(f'The 5 highest values | Release Date | {i}\n', df_copy[['Release_Date', f'{i}']].value_counts().head(5), '\n')
    print(f'The 5 lowest values | Release Date | {i}\n', df_copy[['Release_Date', f'{i}']].value_counts().tail(5), '\n')